# Migration WorkFlow Example

This is a more complex scenario that describes a workflow where there are two service engine groups that need to be merged into a single one. We will define a **TARGET** service engine group that will be used to receive the configuration extracted from the **SOURCE** service engine group. 

<img src="images/migration_scenario.png">


- STEP 1.- Read Source SE Group information

- STEP 2.- Select interface to be migrated from source SE

- STEP 3.- Creating Cloned objects at target SE Group 
   - VRF_NEW
   - NETWORK_NEW
   - NETWORK_SERVICE_NEW
   - INTERFACE_CONFIG 
   - VSVIPs_NEW
   - POOLS_NEW
   - VIRTUAL_SERVICES_NEW (DISABLED STATE)
   - INTF_CONFIG@TARGET_INTF@CLONED_VRF (DISABLED_STATE)

- STEP 4.- Proceed with Migration 
   - 1 Disabling interfaces at Source SE 
   - 2 Enabling interfaces at Target SE
   - 3 Enabling cloned VSs

- STEP 5.- Verification
          - Possible Roll-Back (optional)

- STEP 6.- Cleaning old configurations and renaming to original names
   - Delete migrated VSs
   - Delete migrated VSVIPs
   - Delete migrated Network Service
   - Delete migrated Network
   - Delete migrated VRF
   - Delete migrated Service Engines
   - Delete migrated Service Engine Group
   - Restoring original names 

## Initial Login

In [6]:
from avi.sdk.avi_api import ApiSession
from requests.packages import urllib3
urllib3.disable_warnings()
import json
import pandas as pd
from IPython.display import display, Image, clear_output
from colorama import Fore, Back, Style

# Import custom libraries
from libs.aux import *
from libs.extract import *
from libs.clone import *
from libs.migrate import *

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:3vzgop51i5n9b64c1f9wc4oul5y00y24


### Select  SOURCE and TARGET Service Engine Group Names using Interactive Menu
Sometimes options might not be displayed. If so, stop the cell execution and restart. As last resort, create variables manually

```
source_segroup = "SEG-SOURCE-MAD-002"
target_segroup = "SEG-TARGET-MAD-001" 
```

In [7]:
# Get SOURCE AND TARGET Service Engine Group Names from the list using menu
query = {
    "fields": "name"
}

resp = api.get("serviceenginegroup", params=query)
resp = json.loads(resp.text)["results"]

menu_options = []
i = 0
for item in resp:
    menu_options.insert(i, "Service Engine Group: "+item["name"])
    i = i +1

menu_title = "Please Select an option for SOURCE Service Engine Group to be migrated:"
selected_option = display_menu_from_list(menu_options, menu_title)
    
if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m as SOURCE")
else:
    print("No valid option was selected.")

source_segroup = selected_option.split(": ")[1]

print()
menu_title = "Please Select an option for TARGET Service Engine Group:"
selected_option = display_menu_from_list(menu_options, menu_title)

if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m as TARGET")
    if selected_option.split(": ")[1] == source_segroup:
        while selected_option.split(": ")[1] == source_segroup:
          print()
          print (" !!!!! SOURCE AND TARGET Service Engine Groups cannot be the same !!!!! Repeat Selection for TARGET...")
          menu_title = "Please Select an different option for TARGET Service Engine Group:"
          selected_option = display_menu_from_list(menu_options, menu_title)
          print(selected_option)
else:
    print("No valid option was selected.")

target_segroup = selected_option.split(": ")[1] 

Please Select an option for SOURCE Service Engine Group to be migrated:
--------------------------------------------------------------
1. Service Engine Group: Default-Group
2. Service Engine Group: SEG-SOURCE-MAD-002
3. Service Engine Group: SEG-TARGET-MAD-001
You selected: Service Engine Group: SEG-SOURCE-MAD-002 as SOURCE

Please Select an option for TARGET Service Engine Group:
--------------------------------------------------------------
1. Service Engine Group: Default-Group
2. Service Engine Group: SEG-SOURCE-MAD-002
3. Service Engine Group: SEG-TARGET-MAD-001
You selected: Service Engine Group: SEG-TARGET-MAD-001 as TARGET


In [ ]:
# Set SOURCE AND TARGET Service Engine Group Names from the list above
# Uncomennt and populate both variables if interactive menu does not work
# source_segroup = "SEG-SOURCE-MAD-002"
# target_segroup = "SEG-TARGET-MAD-001"

## STEP 1.- Collecting Source Service Engine Groups and associated Service Engines Information

The first step is to gather SE Group Information
- Service Engine Group Configuration
- Service Engine Configuration

In [8]:
# Extracting source info
print("\033[1mExtracting SOURCE Service Engine Group Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
source_segroup_data = extract_segroup_data(api, source_segroup)
source_se_data = extract_se_data_from_segroup(api, source_segroup_data)

# Extracting target info
print("\033[1mExtracting TARGET Service Engine Group Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
target_segroup_data = extract_segroup_data(api, target_segroup)
target_se_data = extract_se_data_from_segroup(api, target_segroup_data)

Extracting SOURCE Service Engine Group Information
--------------------------------------------------
The following serviceenginegroup names has been found:
['SEG-SOURCE-MAD-002']

The following serviceengine names has been found:
['192.168.1.22', '192.168.1.23']

Extracting TARGET Service Engine Group Information
--------------------------------------------------
The following serviceenginegroup names has been found:
['SEG-TARGET-MAD-001']

The following serviceengine names has been found:
['192.168.1.20', '192.168.1.21']



## STEP 2.- Explore SOURCE collected information and select the source interface to be migrated

In [9]:
# Exploring Interface Acapters Information of SOURCE SE
source_if_names=[]
for index in range(len(source_se_data)):
    print(Fore.BLUE + Style.BRIGHT +"Showing information of the SOURCE Service Engine "+str(index+1))
    print("--------------------------------------------------"+ Style.RESET_ALL)
    se_data_vnics = source_se_data[index]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    menu_options=[]
    for i in range(len(se_data_vnics)):
        if "vnic_networks" in se_data_vnics[i]:
            if_name = se_data_vnics[i]["if_name"]
            vrf_name = se_data_vnics[i]["vrf_ref"].split("#")[1]
            mac_address = se_data_vnics[i]["mac_address"]
            ip_addr = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["addr"]
            type = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["type"]
            mask = se_data_vnics[i]["vnic_networks"][0]["ip"]["mask"]
            mode = se_data_vnics[i]["vnic_networks"][0]["mode"]
            menu_option = "Interface "+ if_name+" at VRF "+vrf_name+" with mac "+ mac_address+" and IP"+type+" ADDRESS "+ip_addr+"/"+str(mask)
            menu_options.append(menu_option)
    menu_options.sort()
    menu_title = "Please Select a candidate interface for SOURCE Service Engine "+str(index+1)+" from above list to be migrated:"
    print()
    selected_option = None
    while selected_option == None:
        selected_option = display_menu_from_list(menu_options, menu_title)
        if selected_option:
           selected_if_name = selected_option.split(" ")[1]
           print()
           print(Fore.RED + "You selected: "+ Style.BRIGHT + selected_if_name + Style.NORMAL +" as SOURCE interface for Service Engine "+str(index+1))
           print(Style.RESET_ALL)
           source_if_names.insert(index,selected_if_name)
        else:
           clear_output


Showing information of the SOURCE Service Engine 1
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,enabled,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:83:62,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:83:6c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:83:76,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:83:80,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:83:8a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,False,"[{'ip': {'ip_addr': {'addr': '192.168.2.22', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:83:94,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:83:9e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:83:a8,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:83:b2,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN



Please Select a candidate interface for SOURCE Service Engine 1 from above list to be migrated:
--------------------------------------------------------------
1. Interface eth1 at VRF vrf_source_mad_002 with mac 00:0c:29:a7:83:8a and IPV4 ADDRESS 192.168.2.22/24

You selected: eth1 as SOURCE interface for Service Engine 1

Showing information of the SOURCE Service Engine 2
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,enabled,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:61:16,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:61:20,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:61:2a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:61:34,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:61:3e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,False,"[{'ip': {'ip_addr': {'addr': '192.168.2.23', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:61:48,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:61:52,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:61:5c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:61:66,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN



Please Select a candidate interface for SOURCE Service Engine 2 from above list to be migrated:
--------------------------------------------------------------
1. Interface eth1 at VRF vrf_source_mad_002 with mac 00:0c:29:a7:61:3e and IPV4 ADDRESS 192.168.2.23/24

You selected: eth1 as SOURCE interface for Service Engine 2



In [ ]:
# source interfaces should be selected using above interactive menu
# Uncomment and populate list variable "source_if_names" if interactive menu does not work using this sample format:
# Typically both names should be the same from a given SE HA pair
# source_if_names = ["eth1", "eth1"]

## STEP 3.- Explore TARGET adapters collected information and select the source interface to be migrated

In [10]:
# Exploring Interface Adapters Information of TARGET SEs
target_if_names=[]
for index in range(len(target_se_data)):
    print("\033[1mShowing information of the TARGET Service Engine "+str(index+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = target_se_data[index]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    menu_options=[]
    for i in range(len(se_data_vnics)):
        if "vnic_networks" not in se_data_vnics[i]:
            if_name = se_data_vnics[i]["if_name"]
            vrf_name = se_data_vnics[i]["vrf_ref"].split("#")[1]
            #mac_address = se_data_vnics[i]["mac_address"]
            #ip_addr = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["addr"]
            #type = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["type"]
            #mask = se_data_vnics[i]["vnic_networks"][0]["ip"]["mask"]
            #mode = se_data_vnics[i]["vnic_networks"][0]["mode"]
            menu_option = "Interface "+ if_name+" at VRF "+vrf_name+" has no current IP information and it appears available "
            menu_options.append(menu_option)
    menu_options.sort()
    menu_title = "Please Select a candidate interface for TARGET Service Engine "+str(index+1)+" from above list to receive new config:"
    print()
    selected_option = None
    while selected_option == None:
        selected_option = display_menu_from_list(menu_options, menu_title)
        if selected_option:
            selected_if_name = selected_option.split(" ")[1]
            print()
            print(Fore.RED + "You selected: "+ Style.BRIGHT + selected_if_name + Style.NORMAL +" as TARGET interface for Service Engine "+str(index+1))
            print(Style.RESET_ALL)
            target_if_names.insert(index,selected_if_name)
        else:
            clear_output

Showing information of the TARGET Service Engine 1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth9,False,eth5,00:0c:29:c4:a7:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth3,False,eth3,00:0c:29:c4:a7:b9,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth4,False,eth6,00:0c:29:c4:a7:c3,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth6,False,eth8,00:0c:29:c4:a7:cd,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth8,False,eth1,00:0c:29:c4:a7:d7,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,False,eth1,False,eth4,00:0c:29:c4:a7:e1,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.40', '..."
6,Unknown,True,False,eth2,False,eth7,00:0c:29:c4:a7:eb,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth5,False,eth9,00:0c:29:c4:a7:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth7,False,eth2,00:0c:29:c4:a7:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for TARGET Service Engine 1 from above list to receive new config:
--------------------------------------------------------------
1. Interface eth2 at VRF global has no current IP information and it appears available 
2. Interface eth3 at VRF global has no current IP information and it appears available 
3. Interface eth4 at VRF global has no current IP information and it appears available 
4. Interface eth5 at VRF global has no current IP information and it appears available 
5. Interface eth6 at VRF global has no current IP information and it appears available 
6. Interface eth7 at VRF global has no current IP information and it appears available 
7. Interface eth8 at VRF global has no current IP information and it appears available 
8. Interface eth9 at VRF global has no current IP information and it appears available 

You selected: eth2 as TARGET interface for Service Engine 1

Showing information of the TARGET Service Engine 2 
---------------

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth6,False,eth8,00:0c:29:ba:47:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth8,False,eth1,00:0c:29:ba:47:13,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth1,False,eth4,00:0c:29:ba:47:1d,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.41', '..."
3,Unknown,True,False,eth2,False,eth7,00:0c:29:ba:47:27,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth5,False,eth9,00:0c:29:ba:47:31,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,False,eth7,False,eth2,00:0c:29:ba:47:3b,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth9,False,eth5,00:0c:29:ba:47:45,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth3,False,eth3,00:0c:29:ba:47:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth4,False,eth6,00:0c:29:ba:47:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for TARGET Service Engine 2 from above list to receive new config:
--------------------------------------------------------------
1. Interface eth2 at VRF global has no current IP information and it appears available 
2. Interface eth3 at VRF global has no current IP information and it appears available 
3. Interface eth4 at VRF global has no current IP information and it appears available 
4. Interface eth5 at VRF global has no current IP information and it appears available 
5. Interface eth6 at VRF global has no current IP information and it appears available 
6. Interface eth7 at VRF global has no current IP information and it appears available 
7. Interface eth8 at VRF global has no current IP information and it appears available 
8. Interface eth9 at VRF global has no current IP information and it appears available 

You selected: eth2 as TARGET interface for Service Engine 2



In [ ]:
# interfaces should be selected using above interactive menu
# Uncomment and populate list variable "target_if_names" if interactive menu does not work using this sample format:
# Typically both names should be the same from a given SE HA pair
# target_if_names = ["eth2", "eth2"]

## STEP 4.- Extract information from selected interfaces

In [11]:
interface_config = extract_interface_data(api, source_se_data, source_if_names, target_se_data, target_if_names)
output_config = interface_config

print()
print("\033[1mExtracting SOURCE Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
source_vs_data = extract_vss_data(api, source_se_data, interface_config)
# Update output_config variable with extracted info
output_config["virtualservices"] = source_vs_data

print
print("\033[1mExtracting SOURCE VSVIPs and Related Network Placement Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

network_vsvips_config = extract_network_vsvip_data(api, source_vs_data)
# Update variable with extracted info
output_config["network_config"]=network_vsvips_config["network_config"]
output_config["vsvips_config"]=network_vsvips_config["vsvips_config"]

Extracting information from SOURCE SE 1 interface to migrate eth1 
----------------------------------------------------------------------------------------------------
   - Found IP Address 192.168.2.22/24
   - Found MAC Address 00:0c:29:a7:83:8a
   - Found VRF Context vrf_source_mad_002

Extracting information from SOURCE SE 2 interface to migrate eth1 
----------------------------------------------------------------------------------------------------
   - Found IP Address 192.168.2.23/24
   - Found MAC Address 00:0c:29:a7:61:3e
   - Found VRF Context vrf_source_mad_002

Extracting IP Routing Information for the VRF Context vrf_source_mad_002 where eth1 is attached to 
----------------------------------------------------------------------------------------------------
   - Found static IPV4 route prefix 192.168.0.0/16 pointing to next-hop --> 192.168.2.10
   - Found static IPV4 route prefix 0.0.0.0/0 pointing to next-hop --> 192.168.2.1

Extracting Networkservice Information for the 

In [12]:
print (" Showing full extracted config...")
print(json.dumps(output_config, indent=3, sort_keys=False))

 Showing full extracted config...
{
   "ip_routing_config": {
      "vrfcontext": "vrf_source_mad_002",
      "vrf_ref": "https://192.168.1.15/api/vrfcontext/vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2#vrf_source_mad_002",
      "vrf_uuid": "vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2",
      "static_routes": [
         {
            "route_type": "V4",
            "route_prefix": "192.168.0.0",
            "route_mask": 16,
            "route_next_hop": "192.168.2.10"
         },
         {
            "route_type": "V4",
            "route_prefix": "0.0.0.0",
            "route_mask": 0,
            "route_next_hop": "192.168.2.1"
         }
      ]
   },
   "se_pairs": [
      {
         "source_se": {
            "se_uuid": "se-564d63c6-3882-ca52-7be5-b04c9ba78358",
            "se_name": "192.168.1.22",
            "if_name": "eth1",
            "if_vrf_ref": "https://192.168.1.15/api/vrfcontext/vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2#vrf_source_mad_002",
         

## STEP 4.- Create cloned objects from selected configuration

 - Clone VRF
 - Clone Placement Networks
 - Clone Network Services (if any)
 - Clone VSVIPs
 - Clone VirtualServices (set in disabled state)

In [13]:
# 1 Clone existing VRF object
# New objects will take a name in the form <original_name>-NEW
# Returns a new dictionary with cloned object 
source_vrf = output_config["ip_routing_config"]["vrfcontext"]
cloned_vrf = clone_vrf (api, source_vrf)
print()
print("\033[1mCheck returned output for further verification:\033[0m")
print("\033[1m-----------------------------------------------\033[0m")
print(json.dumps(cloned_vrf, indent=3, sort_keys=True))

Cloning existing vrfcontext vrf_source_mad_002 into vrf_source_mad_002-NEW
<Response [201]>
- New vrfcontext named vrf_source_mad_002-NEW CREATED

Check returned output for further verification:
-----------------------------------------------
{
   "_last_modified": "1725440899166594",
   "cloud_ref": "https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1#Default-Cloud",
   "debugvrfcontext": {
      "command_buffer_interval": 2,
      "command_buffer_size": 32768
   },
   "internal_gateway_monitor": {
      "disable_gateway_monitor": false,
      "gateway_monitor_failure_threshold": 10,
      "gateway_monitor_interval": 1000,
      "gateway_monitor_success_threshold": 15
   },
   "lldp_enable": true,
   "name": "vrf_source_mad_002-NEW",
   "static_routes": [
      {
         "next_hop": {
            "addr": "192.168.2.10",
            "type": "V4"
         },
         "prefix": {
            "ip_addr": {
               "addr": "192.168.0.0",
               "type": 

In [14]:
# 2 Clone network object in the previously cloned VRF
# New objects will take a name in the form <original_name>-NEW
# Returns a new dictionary with cloned object 
target_vrf = cloned_vrf["name"]
source_networks = output_config["network_config"]
cloned_networks = []
for source_network in source_networks:
  cloned_network = clone_network (api, source_network, target_vrf)
  cloned_networks.append(cloned_network)
print()
print("\033[1mCheck returned output for further verification:\033[0m")
print("\033[1m-----------------------------------------------\033[0m")
print(json.dumps(cloned_networks, indent=3, sort_keys=True))

Cloning existing network network_source_mad_002 into network_source_mad_002-NEW at VRF vrf_source_mad_002-NEW
<Response [201]>
- New network named network_source_mad_002-NEW CREATED

Check returned output for further verification:
-----------------------------------------------
[
   {
      "_last_modified": "1725440902845369",
      "cloud_ref": "https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1#Default-Cloud",
      "configured_subnets": [
         {
            "prefix": {
               "ip_addr": {
                  "addr": "192.168.2.0",
                  "type": "V4"
               },
               "mask": 24
            },
            "static_ip_ranges": [
               {
                  "range": {
                     "begin": {
                        "addr": "192.168.2.50",
                        "type": "V4"
                     },
                     "end": {
                        "addr": "192.168.2.79",
                        "type": "V4"


In [15]:
# 3 Clone existing networkservice object in the previously cloned VRF in the target service engine group. 
# Returns a new dictionary with cloned object configuration 

if output_config["network_service_config"]:
    source_networkservice = output_config["network_service_config"]["name"]
    cloned_networkservice = clone_networkservice (api, source_networkservice, target_vrf, target_segroup )
    print()
    print("\033[1mCheck returned output for further verification:\033[0m")
    print("\033[1m-----------------------------------------------\033[0m")
    print(json.dumps(cloned_networkservice, indent=3, sort_keys=True))
else:
    print("\033[1mNo existing NetworkService to clone :\033[0m")
    print("\033[1m-------------------------------------\033[0m")

Cloning existing networkservice NS-SOURCE-MAD-002 into NS-SOURCE-MAD-002-NEW
<Response [201]>
- New networkservice named NS-SOURCE-MAD-002-NEW CREATED

Check returned output for further verification:
-----------------------------------------------
{
   "_last_modified": "1725440907934959",
   "cloud_ref": "https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1#Default-Cloud",
   "name": "NS-SOURCE-MAD-002-NEW",
   "routing_service": {
      "advertise_backend_networks": false,
      "enable_auto_gateway": false,
      "enable_routing": true,
      "enable_vip_on_all_interfaces": true,
      "enable_vmac": false,
      "floating_intf_ip": [
         {
            "addr": "192.168.2.30",
            "type": "V4"
         },
         {
            "addr": "192.168.2.31",
            "type": "V4"
         }
      ],
      "graceful_restart": false,
      "routing_by_linux_ipstack": false
   },
   "se_group_ref": "https://192.168.1.15/api/serviceenginegroup/serviceengineg

In [16]:
# 4 Clone existing VSVIPs objects extracted from a source VRF into the previously cloned VRF into the target service engine group. 
# Returns a" new dictionary with cloned object configuration

cloned_vsvips=[]
for vsvip in output_config["vsvips_config"]:
    target_network = vsvip["placement_network"]+"-NEW"
    vsvip_name = vsvip["name"]
    cloned_vsvip = clone_vsvips(api, vsvip_name, target_vrf, target_network)
    cloned_vsvips.append(cloned_vsvip)
if (cloned_vsvips):
    print("\033[1mNew cloned objects summary for network"+target_network+"\033[0m")
    print("\033[1m------------------------------------------------------\033[0m")
    for item in cloned_vsvips:
      print ("VSVIP named \033[1m"+item["name"]+"\033[0m with IP Address \033[1m"+item["vip"][0]["ip_address"]["addr"]+"\033[0m at target VRF \033[1m"+ target_vrf+"\033[0m")
print()
print("\033[1mCheck full returned output for further verification:\033[0m")
print("\033[1m----------------------------------------------------\033[0m")
print(json.dumps(cloned_vsvips, indent=3, sort_keys=True))

Cloning existing vsvip vsvip-source-151 into new vsvip vsvip-source-151-NEW @ VRF context vrf_source_mad_002-NEW
- New vsvip named vsvip-source-151-NEW CREATED

Cloning existing vsvip vsvip-source-152 into new vsvip vsvip-source-152-NEW @ VRF context vrf_source_mad_002-NEW
- New vsvip named vsvip-source-152-NEW CREATED

New cloned objects summary for networknetwork_source_mad_002-NEW
------------------------------------------------------
VSVIP named vsvip-source-151-NEW with IP Address 192.168.1.151 at target VRF vrf_source_mad_002-NEW
VSVIP named vsvip-source-152-NEW with IP Address 192.168.1.152 at target VRF vrf_source_mad_002-NEW

Check full returned output for further verification:
----------------------------------------------------
[
   {
      "_last_modified": "1725440911630936",
      "cloud_ref": "https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1#Default-Cloud",
      "east_west_placement": false,
      "name": "vsvip-source-151-NEW",
      "tenant_re

In [17]:
# 5 Clone Pools related to an VS existing in a given source VRF and duplicate into a target_vrf 
# Returns a new dictionary with cloned object configuration
cloned_pools = []
for vs in output_config["virtualservices"]:
    vs_name = vs["name"]
    cloned_pool = clone_pools (api, vs_name, target_vrf)
    cloned_pools.append(cloned_pool)
if (cloned_pools):
    print("\033[1mNew cloned objects summary\033[0m")
    print("\033[1m-------------------------\033[0m")
    for pool in cloned_pools:
        print ("Pool named \033[1m"+pool["name"]+"\033[0m with following servers:")
        for server in pool["servers"]:
           print (" - Server IP "+server["ip"]["addr"])
print()
print("\033[1mCheck full returned output for further verification:\033[0m")
print("\033[1m----------------------------------------------------\033[0m")
print(json.dumps(cloned_pools, indent=3, sort_keys=True))

Found Pool Name nginx-003
------------------------------
Cloning existing POOL named nginx-003 attached to VS VS_SOURCE_151 into new POOL named nginx-003-NEW @ VRF context vrf_source_mad_002-NEW
- New pool named nginx-003-NEW CREATED

Found Pool Name nginx-004
------------------------------
Cloning existing POOL named nginx-004 attached to VS VS_SOURCE_152 into new POOL named nginx-004-NEW @ VRF context vrf_source_mad_002-NEW
- New pool named nginx-004-NEW CREATED

New cloned objects summary
-------------------------
Pool named nginx-003-NEW with following servers:
 - Server IP 192.168.1.22
 - Server IP 192.168.1.23
 - Server IP 192.168.1.24
 - Server IP 192.168.1.25
 - Server IP 192.168.1.26
 - Server IP 192.168.1.27
 - Server IP 192.168.1.28
Pool named nginx-004-NEW with following servers:
 - Server IP 192.168.1.55

Check full returned output for further verification:
----------------------------------------------------
[
   {
      "_last_modified": "1725440916318956",
      "analyt

In [18]:
# 6 Clone VirtualServices from a given source_vrf and segroup
# Returns a new dictionary with cloned object configuration
source_vrf=output_config["ip_routing_config"]["vrfcontext"]
#target_vrf=cloned_vrf["name"]
#target_vrf= "vrf_source_mad_002-NEW"
#target_segroup = "SEG-TARGET-MAD-001"
cloned_vss=[]
for source_vs in output_config["virtualservices"]:
    source_vs_name = source_vs["name"]
    cloned_vs = clone_virtualservices (api, source_vs_name, target_vrf, target_segroup)
    cloned_vss.append(cloned_vs)
if (cloned_vss):
    print("\033[1mNew cloned VirtualServices summary\033[0m")
    print("\033[1m-------------------------\033[0m")
    for item in cloned_vss:
        print ("Virtual Service name \033[1m"+item["name"]+"\033[0m with following config:")
        print (" - VRF Context name "+item["vrf_context_ref"].split("#")[1])
        print (" - VSVIP name "+item["vsvip_ref"].split("#")[1])
        print (" - Pool name "+item["pool_ref"].split("#")[1])
print()
print("\033[1mCheck full returned output for further verification:\033[0m")
print("\033[1m----------------------------------------------------\033[0m")
print(json.dumps(cloned_vss, indent=3, sort_keys=True))

 - Found virtual service VS_SOURCE_151
-------------------------------------------------------------------------------
Extracting information for source VS VS_SOURCE_151....
    - Found source VSVIP vsvip-source-151
    - Found source POOL nginx-003

Looking for candidate target objects already cloned
 - Found candidate VSVIP vsvip-source-151-NEW
 - Found candidate Pool nginx-003-NEW

Everything looks OK, cloning VS
 - New virtualservice named VS_SOURCE_151-NEW CREATED

 - Found virtual service VS_SOURCE_152
-------------------------------------------------------------------------------
Extracting information for source VS VS_SOURCE_152....
    - Found source VSVIP vsvip-source-152
    - Found source POOL nginx-004

Looking for candidate target objects already cloned
 - Found candidate VSVIP vsvip-source-152-NEW
 - Found candidate Pool nginx-004-NEW

Everything looks OK, cloning VS
 - New virtualservice named VS_SOURCE_152-NEW CREATED

New cloned VirtualServices summary
---------------

In [ ]:
# This is a multiple step procedure 
# Steps from 4 and beyond affect to service since actually migrate...

## 2.1  Migration Procedure

In [19]:
# 1 Adjust max_vs_per_se parameter
new_max_vs_per_se = adjust_max_vs_per_se (api, source_segroup, target_segroup)
print("Adjusting Maximum Virtual Services per SE parameter to "+ str(new_max_vs_per_se) + " to acommodate migrated VSs")

 - Found 2 virtual services at source Service Engine group SEG-SOURCE-MAD-002
 - Found 6 virtual services at target Service Engine group SEG-SOURCE-MAD-002
Increase the number of VS per SE to 13 to acommodate imported VSs 
<Response [200]>
- Object serviceenginegroup/serviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f named SEG-TARGET-MAD-001 modified OK

Adjusting Maximum Virtual Services per SE parameter to 13 to acommodate migrated VSs


In [20]:
# 2 Disable Interfaces at target SEs
for i in range(len(output_config["se_pairs"])):
  se_name = output_config["se_pairs"][i]["target_se"]["se_name"]
  if_name = output_config["se_pairs"][i]["target_se"]["if_name"]
  disable_interface(api, se_name, if_name)

 - Interface eth2 set to DISABLED at SE 192.168.1.20
 - Interface eth2 set to DISABLED at SE 192.168.1.21


In [21]:
# 3  Apply IP Address configuration at target SEs
target_vrf = cloned_vrf["name"]
for i in range(len(output_config["se_pairs"])):
  target_se_data = output_config["se_pairs"][i]["target_se"]
  configure_interface(api, target_se_data, target_vrf)

 - Interface eth2 of Service Engine 192.168.1.20 configured with IP Address 192.168.2.22/24 at VRF vrf_source_mad_002-NEW
 - Interface eth2 of Service Engine 192.168.1.21 configured with IP Address 192.168.2.23/24 at VRF vrf_source_mad_002-NEW


In [22]:
# !!!! WARNING... this affect service
# 4 Disable Interfaces at source SEs
for i in range(len(output_config["se_pairs"])):
  se_name = output_config["se_pairs"][i]["source_se"]["se_name"]
  if_name = output_config["se_pairs"][i]["source_se"]["if_name"]
  disable_interface(api, se_name, if_name)

 - Interface eth1 set to DISABLED at SE 192.168.1.22
 - Interface eth1 set to DISABLED at SE 192.168.1.23


In [23]:
# 5 Disable source Virtual Services 
for i in range(len(output_config["virtualservices"])):
  vs_name = output_config["virtualservices"][i]["name"]
  disable_vs (api, vs_name)

 - VirtualService VS_SOURCE_151 set to DISABLED
 - VirtualService VS_SOURCE_152 set to DISABLED


In [24]:
# 6 Enable target interfaces
for i in range(len(output_config["se_pairs"])):
  se_name = output_config["se_pairs"][i]["target_se"]["se_name"]
  if_name = output_config["se_pairs"][i]["target_se"]["if_name"]
  enable_interface(api, se_name, if_name)

 - Interface eth2 set to ENABLED at SE 192.168.1.20
 - Interface eth2 set to ENABLED at SE 192.168.1.21


In [25]:
# 7 Enable target (cloned) Virtual Services 
for vs in cloned_vss:
  vs_name = vs["name"]
  enable_vs (api, vs_name)

 - VirtualService VS_SOURCE_151-NEW set to ENABLED
 - VirtualService VS_SOURCE_152-NEW set to ENABLED
